#### Cross Encoders

`Similarity Calculation`: Cross Encoders calculate similarity scores by taking pairs of texts as input and directly computing their similarity. Unlike Sentence Transformers, which encode texts separately, Cross Encoders consider both texts simultaneously.

`Performance`: Cross Encoders generally provide superior performance in terms of accuracy because they capture the interactions between the two texts more effectively. This makes them especially good for tasks where precise semantic matching is crucial.

`Computational Cost`: Cross Encoders are often slower than Sentence Transformers because they need to process each pair of texts individually. This results in higher computational costs, especially when comparing many pairs of texts.

`Use Case` - Re-ranking: Due to their higher computational cost, Cross Encoders are commonly used to re-rank the top-k results from a Sentence Transformer model. The typical workflow involves:

    - Using a Sentence Transformer to encode a large corpus and quickly retrieve the top-k most similar texts.
    - Applying a Cross Encoder to the top-k pairs to re-rank them more accurately.

In [2]:
from sentence_transformers import CrossEncoder

In [3]:
# 1. Load a pre-trained CrossEncoder model
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

D:\Anaconda-16-FEB\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# 2. Predict scores for a pair of sentences
scores = model.predict([
    ("How many people live in Berlin?", "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers."),
    ("How many people live in Berlin?", "Berlin is well known for its museums."),
])

scores

array([ 8.6071415, -4.320076 ], dtype=float32)

In [5]:
# 3. Rank a list of passages for a query
query = "How many people live in Berlin?"

passages = [
    "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.",
    "Berlin is well known for its museums.",
    "In 2014, the city state Berlin had 37,368 live births (+6.6%), a record number since 1991.",
    "The urban area of Berlin comprised about 4.1 million people in 2014, making it the seventh most populous urban area in the European Union.",
    "The city of Paris had a population of 2,165,423 people within its administrative city limits as of January 1, 2019",
    "An estimated 300,000-420,000 Muslims reside in Berlin, making up about 8-11 percent of the population.",
    "Berlin is subdivided into 12 boroughs or districts (Bezirke).",
    "In 2015, the total labour force in Berlin was 1.85 million.",
    "In 2013 around 600,000 Berliners were registered in one of the more than 2,300 sport and fitness clubs.",
    "Berlin has a yearly total of about 135 million day visitors, which puts it in third place among the most-visited city destinations in the European Union.",
]

In [6]:
ranks = model.rank(query, passages)

In [7]:
# Print the scores
print("Query:", query)
for rank in ranks:
    print(f"{rank['score']:.2f}\t{passages[rank['corpus_id']]}")

Query: How many people live in Berlin?
8.92	The urban area of Berlin comprised about 4.1 million people in 2014, making it the seventh most populous urban area in the European Union.
8.61	Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.
8.24	An estimated 300,000-420,000 Muslims reside in Berlin, making up about 8-11 percent of the population.
7.60	In 2014, the city state Berlin had 37,368 live births (+6.6%), a record number since 1991.
6.35	In 2013 around 600,000 Berliners were registered in one of the more than 2,300 sport and fitness clubs.
5.42	Berlin has a yearly total of about 135 million day visitors, which puts it in third place among the most-visited city destinations in the European Union.
3.45	In 2015, the total labour force in Berlin was 1.85 million.
0.33	Berlin is subdivided into 12 boroughs or districts (Bezirke).
-4.24	The city of Paris had a population of 2,165,423 people within its administrative city limits as of Jan

#### How to Use a Cross Encoder

1. **Input Concatenation:**
   - Combine the two inputs (e.g., a query and a document, or two sentences) into a single sequence, typically separated by a special token (like `[SEP]` in BERT models).

2. **Encoding:**
   - Pass the concatenated input sequence through the cross encoder model, which jointly processes both inputs and generates a representation for the combined sequence.

3. **Prediction:**
   - The model's output is used to predict the relationship between the two inputs, such as a similarity score, classification label, or relevance score.

#### When to Use a Cross Encoder

- **Detailed Pairwise Comparison:**
  Use cross encoders when you need to capture detailed interactions between two inputs, making them ideal for tasks like sentence pair classification, paraphrase detection, and semantic matching.

- **Ranking and Classification:**
  Cross encoders are highly effective in ranking tasks where the precise relationship between a query and potential candidates needs to be evaluated, as well as in binary or multi-class classification tasks involving input pairs.

- **High-Accuracy Requirements:**
  When accuracy is critical, especially in tasks where understanding subtle nuances between two inputs is necessary, cross encoders are preferred due to their ability to model complex interactions.

#### Strengths of Cross Encoders

- **Rich Interaction Modeling:**
  Cross encoders capture detailed, token-level interactions between inputs, leading to a more accurate and nuanced understanding of their relationship.

- **High Accuracy:**
  By processing both inputs together, cross encoders often achieve higher accuracy on tasks requiring precise pairwise comparisons.

- **Versatility:**
  Cross encoders can be applied to various NLP tasks, including sentence similarity, paraphrase identification, and document ranking.

#### Limitations of Cross Encoders

- **Computationally Expensive:**
  Cross encoders are more computationally intensive than dual encoders, as each pair of inputs must be processed together, leading to slower inference times.

- **Scalability Issues:**
  The joint encoding of input pairs limits the scalability of cross encoders, making them less efficient for large-scale applications where many pairs need to be processed rapidly.

- **Inefficient for Large-Scale Retrieval:**
  In scenarios like document retrieval, where a query must be compared against numerous candidates, cross encoders can be inefficient since they require encoding every possible pair from scratch.
